In [78]:
import binascii
import datetime
from lxml import etree, objectify
from xml.etree import ElementTree as ET
import pathlib
import struct
from collections import Counter, defaultdict

from mechaparabellum.utils import get_newest

In [36]:
DIR = pathlib.Path('C:/Program Files (x86)/Steam/steamapps/common/Mechabellum/ProjectDatas/Replay')
OUT_DIR = pathlib.Path('c:/Users/k/p/mechaparabellum/replays')
STATIC = 0x151
DATE_END = 0x166
XML_START = 0x18a
START_TAG = b'<BattleRecord xmlns'
END_TAG = b'</BattleRecord>'

In [35]:
print(datetime.datetime.now(datetime.UTC).timestamp())
BYTES = defaultdict(Counter)
replay = get_newest(DIR.glob('*.grbr'))
with replay.open('rb') as f:
    f.seek(DATE_END)
    raw = f.read(XML_START - STATIC)
    data = raw.split(b'\x5e\x06\x00\x00', 1)[-1]
    print(int.from_bytes(data[0x12:0x12+6], byteorder='big'))
    for i, b in enumerate(data):
        BYTES[i][b] += 1

1753715190.506257
7696581985280


In [39]:
for replay in DIR.glob('*.grbr'):
    s = replay.read_bytes()
    xml_start = s.find(START_TAG)
    xml_end = s.find(END_TAG)
    xml = s[xml_start:xml_end+len(END_TAG)].decode('utf-8')
    br = objectify.fromstring(xml)
    id_ = br.BattleInfo.BattleID
    fn = (OUT_DIR / str(id_)).with_suffix('.xml')
    fn.write_text(xml)

In [109]:
xml_file = (OUT_DIR / '20250728--134516141.xml')
xml_file = (OUT_DIR / '20250728--201625477.xml')
# br = objectify.fromstring(xml.read_text())
tree = etree.parse(xml_file, parser=etree.XMLParser(recover=True))
root = tree.getroot()

In [110]:
def to_str(elem):
    return ET.tostring(elem).decode('utf-8')

In [139]:
prs = root.find('playerRecords')
print(prs[0].find('name').text)
[t.tag for t in prs[0]]
for round in prs[0].find('playerRoundRecords'):
    print(f'\n\n --- Round: {round.find("round").text}')
    unlocked_units = round.xpath('playerData/shop/unlockedUnits/int')
    actions = round.xpath('actionRecords/MatchActionData')
    for unit in unlocked_units:
        print(f' Available: {unit.text}')
    for action in actions:
        act_name = action.attrib.values()[0]
        try:
            match act_name:
                case 'PAD_ChooseAdvanceTeam':
                    uid = action.find('ID').text
                    idx = action.find('Index').text
                    print(f'Choose starting setup: #{idx}: {uid}')
                case 'PAD_UpgradeTechnology':
                    unit = int(action.find('UID').text)
                    tech  = int(action.find('TechID').text)
                    print(f'Upgrade technology: {unit} - {tech}')
                case 'PAD_ActiveEnergyTowerSkill':
                    skill = int(action.find('SkillID').text)
                    print(f'Active energy to tower skill: {skill}')
                case 'PAD_BuyUnit':
                    uid = action.find('UID').text
                    print(f'Buy unit: {uid}')
                case 'PAD_UnlockUnit':
                    uid = action.find('UID').text
                    print(f'Unlock unit: {uid}')
                case 'PAD_UpgradeUnit':
                    uid = action.find('UID').text
                    idx = action.find('UIDX').text
                    print(f'Upgrade unit: {idx}')
                case 'PAD_FinishDeploy':
                    print('Finish deploy')
                case 'PAD_Undo':
                    print('Undo')
                case 'PAD_ReleaseCommanderSkill':
                    skill = int(action.find('ID').text)
                    index = int(action.find('SkillIndex').text)
                    print(f'Release commander skill: {skill}')
                case 'PAD_MoveUnit':
                    moves = action.xpath('moveUnitDatas/MoveUnitData')
                    for data in moves:
                        x = data.find('position').find('x').text
                        y = data.find('position').find('y').text
                        uid = data.find('unitID').text
                        index = data.find('unitIndex').text
                        print(f'Move unit: {uid} #{index} x: {x} y: {y}')
                case 'PAD_ChooseReinforceItem':
                    idx = action.find('Index').text
                    uid = action.find('ID').text
                    print(f'Choose reinforce item: {uid} ({idx})')
                case 'PAD_UseEquipment':
                    uid = action.find('EquipmentID').text
                    idx = action.find('UnitIndex').text
                    print(f'Use equipment: {uid} on unit #{idx}')
                case 'PAD_GiveUp':
                    print('Surrender')
                case _:
                    print(to_str(action))
        except AttributeError:
            print(to_str(action))


Kimvais


 --- Round: 0
Choose starting setup: #2: 9894


 --- Round: 1
 Available: 21
 Available: 15
Unlock unit: 10
Buy unit: 10
Buy unit: 10
Move unit: 21 #0 x: -175 y: -75
Move unit: 21 #1 x: -85 y: -75
Move unit: 15 #2 x: -220 y: -80
Move unit: 15 #3 x: -130 y: -80
Move unit: 15 #4 x: -40 y: -80
Move unit: 10 #5 x: -140 y: -205
Move unit: 10 #6 x: -80 y: -115
Finish deploy


 --- Round: 2
 Available: 21
 Available: 15
 Available: 10
Choose reinforce item: 31301 (2)
Active energy to tower skill: 3
Unlock unit: 13
Buy unit: 13
Move unit: 13 #7 x: 140 y: -135
Release commander skill: 1100001
Upgrade unit: 0
Buy unit: 21
Move unit: 21 #8 x: 5 y: -75
Undo
Undo
Undo
Undo
Buy unit: 13
Undo
Buy unit: 21
Move unit: 21 #8 x: 5 y: -75
Release commander skill: 1100001
Upgrade unit: 8
Finish deploy


 --- Round: 3
 Available: 21
 Available: 15
 Available: 10
 Available: 13
Choose reinforce item: 13030002 (0)
Upgrade unit: 0
Upgrade unit: 8
Use equipment: 13030002 on unit #8
Buy unit: 21
Undo
B

In [96]:
# sorted(BYTES.items(), key=lambda t: len(t[1]))
for idx, b in BYTES.items():
    if len(b) > 1:
        print(f'{idx:02x} {len(b):02d}: {b}')

In [153]:
n = 281474976711080080

In [157]:
n.to_bytes(8, 'big').hex(' ')

'03 e8 00 00 00 06 78 90'